<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/SVM_Exact_Replication_Experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Data Science
import re
import csv
import json
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

# SVM
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report

In [116]:
#NewSpiral is from the new HandPD dataset. The new one also comes with images and such, but we are using the data that has the exact same features from old HandPd dataset, except also includes patient ID.
spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewSpiral.csv')

In [117]:
spiral_df['CLASS_TYPE'].value_counts()

1    140
2    124
Name: CLASS_TYPE, dtype: int64

In [118]:
meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewMeander.csv')

In [119]:
meander_df.columns

Index(['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE', 'RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT'],
      dtype='object')

In [ ]:
spiral_df['ID_PATIENT'].unique().tolist()

In [121]:
spiral_df[spiral_df['ID_PATIENT'] == 31]

,_ID_EXAM,IMAGE_NAME,ID_PATIENT,CLASS_TYPE,GENDER,RIGH/LEFT-HANDED,AGE,RMS,MAX_BETWEEN_ET_HT,MIN_BETWEEN_ET_HT,STD_DEVIATION_ET_HT,MRT,MAX_HT,MIN_HT,STD_HT,CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT
141,P2,mea1-P2.jpg,31,2,F,R,59,3726.488067,5893.347008,37785.94529,0.306602,24.403343,179.554888,0.008474,1739.851476,0.267742
164,P26,mea1-P26.jpg,31,2,F,L,59,4587.166408,6561.231877,37033.96196,0.018378,22.328549,184.806500,0.057158,1388.315960,0.266871
172,P2,mea2-P2.jpg,31,2,F,R,59,3469.841548,5282.395889,33582.16916,0.000129,18.103416,170.531514,0.045752,1130.591703,0.204545
195,P26,mea2-P26.jpg,31,2,F,L,59,6210.596735,7365.102809,37813.01828,0.002359,20.958406,160.395258,0.024497,1281.094994,0.128205
203,P2,mea3-P2.jpg,31,2,F,R,59,5066.989293,7463.980609,35307.01259,0.000000,22.643151,176.392744,0.008650,1614.293957,0.246711
226,P26,mea3-P26.jpg,31,2,F,L,59,6544.571924,8612.729092,36154.08597,0.013198,27.237956,180.306815,0.003577,1761.998890,0.243421
234,P2,mea4-P2.jpg,31,2,F,R,59,3748.920825,6607.203671,36244.36135,0.001141,24.606020,179.208863,0.028543,1782.895068,0.263158
257,P26,mea4-P26.jpg,31,2,F,L,59,4921.267695,6948.380564,35304.08954,0.017636,25.699685,182.482156,0.055527,1698.122391,0.235915


From the above, we can see that there are two patients with the ID of 31. So we're going to give one of them an ID of 32 instead, as that ID does not belong to any other.

In [122]:
spiral_df['ID_PATIENT'][spiral_df['_ID_EXAM'] == "P26"] = 32

In [123]:
spiral_df[spiral_df['ID_PATIENT'] == 32]

,_ID_EXAM,IMAGE_NAME,ID_PATIENT,CLASS_TYPE,GENDER,RIGH/LEFT-HANDED,AGE,RMS,MAX_BETWEEN_ET_HT,MIN_BETWEEN_ET_HT,STD_DEVIATION_ET_HT,MRT,MAX_HT,MIN_HT,STD_HT,CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT
164,P26,mea1-P26.jpg,32,2,F,L,59,4587.166408,6561.231877,37033.96196,0.018378,22.328549,184.806500,0.057158,1388.315960,0.266871
195,P26,mea2-P26.jpg,32,2,F,L,59,6210.596735,7365.102809,37813.01828,0.002359,20.958406,160.395258,0.024497,1281.094994,0.128205
226,P26,mea3-P26.jpg,32,2,F,L,59,6544.571924,8612.729092,36154.08597,0.013198,27.237956,180.306815,0.003577,1761.998890,0.243421
257,P26,mea4-P26.jpg,32,2,F,L,59,4921.267695,6948.380564,35304.08954,0.017636,25.699685,182.482156,0.055527,1698.122391,0.235915


##**Preprocessing**

###Normalization

fi' = (fi - avg)/std

In [124]:
# normalization with the paper's method (formula above)
def feature_normalization(df):
  df_dup = df.copy()

  df1 = df_dup[['RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]

  df2 = df_dup[['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE']]
  avg_dev = df1.mad(axis = 0)
  std_dev = df1.std(axis = 0)

  df1 = df1.sub(avg_dev)
  df1 = df1.divide(std_dev)

  return df2, df1

In [125]:
# normalizing X_spiral
X_spiral = pd.concat(feature_normalization(spiral_df), axis = 1)

In [126]:
# normalizing X_meander
X_meander =  pd.concat(feature_normalization(meander_df), axis = 1)

###Data Splitting
The first one uses 75% of the dataset for training purposes and the remaining 25% for the classification phase. However, instead of partitioning the dataset randomly, we created four subsets in order to guarantee that each individual will be represented in the dataset with its 3 spirals/meanders, with the remaining one being used for classification purposes. In this experiment, the spiral- and meander-based datasets are used individually.

In [149]:
def split_data(df, app_df, drop_df):
  df_IDs = df['ID_PATIENT'].unique().tolist()

  for x in df_IDs:
    row1 = df[df['ID_PATIENT'] == x].iloc[0]
    app_df = app_df.append(row1)
    drop_df = drop_df.drop(row1.name)

  return app_df, drop_df

In [150]:
def represented_train_test_split(df):

  test = pd.DataFrame(columns = df.columns)
  train = df.copy()

  # Some patients may have multiple test IDs so we must base this on Patient ID, which can be same for PD and Ctrl groups. So we split them.
  ctrl_df = df.query('CLASS_TYPE == 1')
  pd_df = df.query('CLASS_TYPE == 2')

  # we can append very easily because each of the IDs are different in ctrl_df and pd_df, since we split from X_spiral, which has all unique IDs
  test, train = split_data(ctrl_df, test, train)
  test, train = split_data(pd_df, test, train)

  test = test.reset_index(drop=True)
  train = train.reset_index(drop=True)

  return test, train

In [151]:
spiral_test, spiral_train = represented_train_test_split(X_spiral)

In [153]:
spiral_train.shape

(198, 16)

In [162]:
spiral_test.shape

(66, 16)

In [164]:
spiral_test['ID_PATIENT'].value_counts()

5      2
23     2
14     1
17     1
38     1
      ..
53     1
181    1
305    1
176    1
1      1
Name: ID_PATIENT, Length: 64, dtype: int64

In [163]:
spiral_train['ID_PATIENT'].value_counts()

5      6
23     6
253    3
18     3
301    3
      ..
80     3
78     3
268    3
76     3
1      3
Name: ID_PATIENT, Length: 64, dtype: int64

As can be seen, we end up with only 2 IDs that are duplicated in PD and control patients: 5 and 23.

###Train-Test-Split

In [ ]:
# creating our final dataset with control and PD Spiral patients and control Meander patients
X = pd.concat([X_spiral, X_meander])
y = pd.concat([y_spiral, y_meander])

In [ ]:
# Train-Test Split, 80% train, 20% test, stratification across PD diagnosis (label)
y_label = y.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.2, stratify = y_label)

In [ ]:
X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)

In [ ]:
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

###**Quick Check**

In [ ]:
y_train.value_counts()

Label
2        237
1        115
dtype: int64

In [ ]:
y_test.value_counts()

Label
2        59
1        29
dtype: int64

##**SVM Implementation**

##**Training**

In [ ]:
clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced')
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

To note, it may be useful to look at the weights for this model (see which features are most valuable), but these are only relevant for a linear kernel. RBF kernel does not have relevant/interpretable weights. And of course, linear kernel is not very helpful and good in predicting (51% accuracy). The features are not linearly separable.

In [ ]:
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

In [ ]:
y_pred = pd.Series(y_pred)

##**Results**

In [ ]:
clf.score(X_test, y_test)

0.8636363636363636

In [ ]:
target_names = ['Control', 'PD']
results = classification_report(y_test, y_pred, target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = confusion_matrix(y_test, y_pred)

In [ ]:
results

,precision,recall,f1-score,support
Control,1.000000,0.586207,0.739130,29.000000
PD,0.830986,1.000000,0.907692,59.000000
accuracy,0.863636,0.863636,0.863636,0.863636
macro avg,0.915493,0.793103,0.823411,88.000000
weighted avg,0.886684,0.863636,0.852144,88.000000


In [ ]:
conf_mat

array([[17, 12],
       [ 0, 59]])

In [ ]:
TN, FP, FN, TP = conf_mat.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("TP: ", TP)
print("TN: ", TN)
print("FP: ", FP)
print("FN: ", FN)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)

TP:  59
TN:  17
FP:  12
FN:  0
Sensitivity:  1.0
Specificity:  0.5862068965517241
NPV:  1.0
PPV:  0.8309859154929577


Can we somehow get the weights for the model to learn which features are considered most useful?